In [1]:
#Importing necessary libraries
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
import json
import numpy as np
import pandas as pd
from datasets import load_dataset
import torch
from torch import nn
from scipy.stats import pearsonr
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split

Maybe the 50 dimensions is a little too small and I should start with 128

In [2]:
model = SentenceTransformer('/home/mendu/Thesis/data/musiccaps/embedding_model')

In [3]:
# Load your fine-tuned local model 
model_path = '/home/mendu/Thesis/data/musiccaps/embedding_model'
model = SentenceTransformer(model_path)

In [4]:
# captions = pd.read_csv('/home/mendu/Thesis/data/musiccaps/lp-music-caps_caption_writing.csv', index_col = [0])

In [5]:
msd_dataset = load_dataset('seungheondoh/LP-MusicCaps-MSD')

In [6]:
train = pd.DataFrame(msd_dataset['train'])
# test = pd.DataFrame(msd_dataset['test'])
# valid = pd.DataFrame(msd_dataset['valid'])

In [7]:
sentences = train['caption_writing'].to_list()

## Training a PCA

In [8]:
# Generate embeddings for your sentences using the fine-tuned model
embedded_sentences = model.encode(sentences, show_progress_bar=True, convert_to_numpy=True)

# Number of PCA components (e.g., reduce to 50 dimensions)
num_components = 50
# obj = PCA(n_components=num_components)

# Fit the PCA model to the embedded sentences (this will find the principal components)
# pca_embeddings = obj.fit_transform(embedded_sentences)

Batches:   0%|          | 0/13903 [00:00<?, ?it/s]

In [9]:
# Project the PCA embeddings back to the original space
projected_embeddings = obj.inverse_transform(pca_embeddings)

# Initialize an empty list to store the Pearson correlation coefficients
pearsons_correlations = []

# Calculate Pearson's correlation for each pair of original and projected embeddings
for original, projected in zip(embedded_sentences, projected_embeddings):
    # Compute Pearson's r
    corr, _ = pearsonr(original, projected)
    pearsons_correlations.append(corr)

# If you want to compute a single Pearson's correlation coefficient for all data
# Concatenate all embeddings and compute the correlation
flat_original = embedded_sentences.flatten()
flat_projected = projected_embeddings.flatten()
overall_corr, _ = pearsonr(flat_original, flat_projected)

# print("Pearsons correlation for each embedding pair:", pearsons_correlations)
print("Overall Pearson's correlation:", overall_corr)

NameError: name 'obj' is not defined

In [ ]:
print("Original embeddings shape:", embedded_sentences.shape)
print("PCA-reduced embeddings shape:", pca_embeddings.shape)

Original embeddings shape: (444865, 768)
PCA-reduced embeddings shape: (444865, 50)


## Training an autoencoder

In [10]:
# import torch
# from torch import nn

# class Autoencoder(nn.Module):
#     def __init__(self, input_size, encoding_size):
#         super(Autoencoder, self).__init__()
#         # Encoder
#         self.encoder = nn.Sequential(
#             nn.Linear(input_size, encoding_size),
#             nn.ReLU(True)
#         )
#         # Decoder
#         self.decoder = nn.Sequential(
#             nn.Linear(encoding_size, input_size),
#             nn.Sigmoid()  # Using Sigmoid because embeddings are likely normalized
#         )

#     def forward(self, x):
#         encoded = self.encoder(x)
#         decoded = self.decoder(encoded)
#         return decoded
    

class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_size):
        super(Autoencoder, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(True),
            nn.BatchNorm1d(512),
            nn.Linear(512, 256),
            nn.ReLU(True),
            nn.BatchNorm1d(256),  # Added batch normalization
            nn.Linear(256, 128),  # Added another layer
            nn.ReLU(True),
            nn.Linear(128, encoding_size),  # Adjusted the size of the encoding layer
            nn.ReLU(True)
        )
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(encoding_size, 128),  # Adjusted the size of the decoding layer
            nn.ReLU(True),
            nn.Linear(128, 256),  # Added another layer
            nn.ReLU(True),
            nn.BatchNorm1d(256),  # Added batch normalization
            nn.Linear(256, 512),
            nn.ReLU(True),
            nn.Linear(512, input_size),
            nn.Sigmoid()  # Using Sigmoid because embeddings are likely normalized
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [11]:
embedded_sentences.shape

(444865, 768)

In [12]:
# Convert the NumPy array of embeddings to a PyTorch tensor
embedded_sentences_tensor = torch.tensor(embedded_sentences, dtype=torch.float32)

# Create a dataset and a dataloader
dataset = TensorDataset(embedded_sentences_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

# # Split dataset into training and validation sets
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_set, val_set = random_split(dataset, [train_size, val_size])

# train_loader = DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True)
# val_loader = DataLoader(val_set, batch_size=64, shuffle=False)

In [13]:
# # Convert the NumPy array of embeddings to a PyTorch tensor
# embedded_sentences_tensor = torch.tensor(embedded_sentences, dtype=torch.float32)

# # Create a dataset and a dataloader without labels
# dataset = TensorDataset(embedded_sentences_tensor)

# # Split dataset into training and validation sets
# train_size = int(0.8 * len(dataset))
# val_size = len(dataset) - train_size
# train_set, val_set = random_split(dataset, [train_size, val_size])

# # Create dataloaders without labels
# train_loader = DataLoader(train_set, batch_size=64, shuffle=True, drop_last=True)
# val_loader = DataLoader(val_set, batch_size=64, shuffle=False)


In [14]:
# print(len(dataset))  # Number of samples in the dataset
# print(dataset[0][0].shape)  # Shape of the first sample in the dataset
# print(dataset[0][0].ndim)
# print(embedded_sentences_tensor.shape)


In [15]:
# # Iterate over the batches
# for batch_idx, (data, target) in enumerate(train_loader):
#     # Print the shape of the data in the first batch
#     print(f"Batch {batch_idx}: Data shape: {data.shape}")


In [16]:
# Instantiate the autoencoder
input_size = embedded_sentences.shape[1]
encoding_size = 64  # change this to whatever size you want to encode down to
autoencoder = Autoencoder(input_size=input_size, encoding_size=encoding_size)

# Define loss function and optimizer
# criterion = nn.MSELoss()
criterion = nn.L1Loss()  # MAE
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=1e-3)

In [17]:
input_size

768

In [18]:
epochs = 50  # Set this to the number of epochs to train for
for epoch in range(epochs):
    for data in dataloader:
        inputs = data[0]
        # Forward pass
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/50, Loss: 0.334824800491333
Epoch 2/50, Loss: 0.33555352687835693
Epoch 3/50, Loss: 0.33216968178749084
Epoch 4/50, Loss: 0.33210620284080505
Epoch 5/50, Loss: 0.32818594574928284
Epoch 6/50, Loss: 0.3308216631412506
Epoch 7/50, Loss: 0.33003878593444824
Epoch 8/50, Loss: 0.3279862701892853
Epoch 9/50, Loss: 0.32929566502571106
Epoch 10/50, Loss: 0.3305172026157379
Epoch 11/50, Loss: 0.32799646258354187
Epoch 12/50, Loss: 0.323348730802536
Epoch 13/50, Loss: 0.3270413875579834
Epoch 14/50, Loss: 0.32281219959259033
Epoch 15/50, Loss: 0.3235558271408081
Epoch 16/50, Loss: 0.32261645793914795
Epoch 17/50, Loss: 0.328728586435318
Epoch 18/50, Loss: 0.32509854435920715
Epoch 19/50, Loss: 0.323576420545578
Epoch 20/50, Loss: 0.3259600102901459
Epoch 21/50, Loss: 0.32450369000434875
Epoch 22/50, Loss: 0.330361932516098
Epoch 23/50, Loss: 0.3207489252090454
Epoch 24/50, Loss: 0.3246179521083832
Epoch 25/50, Loss: 0.3260730504989624
Epoch 26/50, Loss: 0.32480618357658386
Epoch 27/50, L

In [19]:
# # Define the EarlyStopping class
# class EarlyStopping:
#     def __init__(self, patience=5, verbose=False):
#         self.patience = patience
#         self.verbose = verbose
#         self.counter = 0
#         self.best_loss = float('inf')
#         self.early_stop = False

#     def __call__(self, val_loss, model):
#         if val_loss < self.best_loss:
#             self.best_loss = val_loss
#             self.counter = 0
#         else:
#             self.counter += 1
#             if self.counter >= self.patience:
#                 self.early_stop = True
#                 if self.verbose:
#                     print("Early stopping")



In [20]:
# # Instantiate the EarlyStopping object
# early_stopping = EarlyStopping(patience=5, verbose=True)

# epochs = 50
# # Training loop with early stopping
# for epoch in range(epochs):
#     # Train the model
#     model.train()  # Set the model to training mode
#     for batch_idx, data in enumerate(train_loader):
#         optimizer.zero_grad()
#         # Unpack the batch
#         embeddings = data[0]  # Assuming the embeddings are in the first element of the tuple
#         output = model(embeddings)
#         loss = criterion(output, embeddings)  # Use embeddings as target since it's an autoencoder
#         loss.backward()
#         optimizer.step()
        
#     # Validate the model
#     model.eval()  # Set the model to evaluation mode
#     val_loss = 0.0
#     with torch.no_grad():
#         for data in val_loader:
#             embeddings = data[0]  # Assuming the embeddings are in the first element of the tuple
#             output = model(embeddings)
#             val_loss += criterion(output, embeddings).item()

#     val_loss /= len(val_loader)

#     print(f'Epoch {epoch+1}/{epochs}, Validation Loss: {val_loss}')

#     # Call early stopping
#     early_stopping(val_loss, model)

#     if early_stopping.early_stop:
#         print("Early stopping")
#         break

- This constant loss means that we should try going smaller in the embedding space, go to 64 or 32. 
- DO early .
- Higher learning rate

In [21]:
# Switch the autoencoder to evaluation mode
autoencoder.eval()

# Process the entire dataset to obtain the decoded (projected) embeddings
encoded_embeddings = autoencoder.encoder(embedded_sentences_tensor).detach().numpy()
decoded_embeddings = autoencoder.decoder(torch.from_numpy(encoded_embeddings)).detach().numpy()

# Calculate Pearson's correlation
from scipy.stats import pearsonr
correlations = np.array([pearsonr(original, decoded)[0] for original, decoded in zip(embedded_sentences, decoded_embeddings)])

# print('Pearson correlation for each embedding pair:', correlations)
print('Mean Pearson correlation:', np.mean(correlations))

Mean Pearson correlation: 0.784859719757847


Change the metric, reconstruction loss

In [22]:
print("Original embeddings shape:", embedded_sentences_tensor.shape)

# Switch autoencoder to evaluation mode
autoencoder.eval()

# Process the entire dataset to obtain the encoded embeddings
encoded_embeddings = autoencoder.encoder(embedded_sentences_tensor).detach()

print("Entire dataset encoded embeddings shape:", encoded_embeddings.shape)

Original embeddings shape: torch.Size([444865, 768])
Entire dataset encoded embeddings shape: torch.Size([444865, 64])


In [ ]:
'''
epochs = 50  # Set this to the number of epochs to train for
for epoch in range(epochs):
    for data in dataloader:
        inputs = data[0]
        # Forward pass
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')
'''